In [5]:
import pandas as pd
from subprocess import Popen, PIPE
# from sleep_utils.schedule import Sleeper, Optimal, Calendar, Event, adjust
# from sleep_utils.sleep_score import sleep_score
import os
import mne
import yasa
import numpy as np
import matplotlib.pyplot as plt

In [6]:
def convert_csv_to_stages(input_csv):
    sample_data = pd.read_csv(input_csv)
    sample_data =sample_data.fillna(0)
    sampling_rate = 256
    ch_names = ['TP9', 'AF7', 'AF8', 'TP10']
    time_series = []
    time_series.append(np.array(sample_data['ch1']))
    time_series.append(np.array(sample_data['ch2']))
    time_series.append(np.array(sample_data['ch3']))
    time_series.append(np.array(sample_data['ch4']))
    time_series = np.array(time_series)
    info = mne.create_info(ch_names,sampling_rate)
    raw = mne.io.RawArray(time_series, info)
    sls = yasa.SleepStaging(raw, eeg_name="TP10", metadata=dict(age=21, male=False))
    y_pred = sls.predict()
    df_pred = pd.DataFrame({'Stage': y_pred})
    sleep_cycles = list(df_pred["Stage"])
    return sleep_cycles

In [7]:
def calculate_sleep_score(input_csv):
    # Read sleep stages from the text file
    sleep_stages = convert_csv_to_stages(input_csv)
    print(sleep_stages)

    # Define weights for different sleep stage metrics
    weights = {
        'dips_n3': 20,
        'falling_deep_sleep': 15,
        'time_each_stage': 15,
        'times_awake': -8,
        'total_sleep_duration': 12,
        'percentage_deep_sleep': 20
    }

    # Count occurrences of sleep stages
    n3_count = sleep_stages.count(3)
    n3_dips = sleep_stages.count(0) - 1  # transitions from non-N3 to N3

    # Find the index of the first occurrence of N3
    first_n3_index = sleep_stages.index(3) if 3 in sleep_stages else len(sleep_stages)

    # Calculate the time spent in each sleep stage
    total_intervals = len(sleep_stages)
    total_minutes = total_intervals / 2
    time_n1 = sleep_stages[:first_n3_index].count(1) / 2
    time_n2 = sleep_stages[:first_n3_index].count(2) / 2
    time_n3 = n3_count / 2
    time_rem = sleep_stages.count(4) / 2

    # Calculate total sleep duration
    total_sleep_duration = time_n1 + time_n2 + time_n3 + time_rem

    # Count the number of awakenings
    num_awakenings = sum(1 for i in range(1, total_intervals) if sleep_stages[i] == 0 and sleep_stages[i - 1] != 0)

    # Calculate percentage of deep sleep
    percentage_deep_sleep = (time_n3 / total_minutes) * 100

    # Calculate the sleep quality score based on the rubric
    sleep_score = (
        (n3_dips * (weights['dips_n3'] / 15)) +
        ((first_n3_index / 2) * (weights['falling_deep_sleep'] / 10)) +
        (((time_n1 / total_minutes) * 100) * (weights['time_each_stage'] / 25)) +
        (((time_n2 / total_minutes) * 100) * (weights['time_each_stage'] / 25)) +
        (((time_n3 / total_minutes) * 100) * (weights['time_each_stage'] / 25)) +
        (((time_rem / total_minutes) * 100) * (weights['time_each_stage'] / 25)) +
        (num_awakenings * weights['times_awake']) +
        ((total_sleep_duration / 2) * (weights['total_sleep_duration'] / 20)) +
        (percentage_deep_sleep * (weights['percentage_deep_sleep'] / 25))
    )

    return sleep_score

In [8]:
current_dir = os.getcwd()
csv_file_path = os.path.join(current_dir, "sleep_eeg.csv")
sleep_score_value = calculate_sleep_score(csv_file_path)
print("Sleep Score:", sleep_score_value)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\dumbl\\Documents\\Research\\Circadia\\sleep_eeg.csv'